In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver import ActionChains
import time
import mysql.connector
import pandas as pd
# Connect Database
db_connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="1234",
    database="redbus"
)
cursor = db_connection.cursor()
print("Database connected Successfully")

# SQL to create table
create_table_query = """
CREATE TABLE IF NOT EXISTS BusDetails(
    id INT PRIMARY KEY AUTO_INCREMENT,
    state VARCHAR(100) DEFAULT 'Andhra Pradesh',
    route_name VARCHAR(255),
    route_link VARCHAR(255),
    bus_name VARCHAR(255),
    bus_type VARCHAR(255),
    departing_time TEXT,
    duration VARCHAR(50),
    reaching_time TEXT,
    star_rating TEXT,
    price VARCHAR(50),
    seat_availability TEXT
);
"""
cursor.execute(create_table_query)
print("Table created successfully")

# Initialize the WebDriver
driver = webdriver.Chrome()
driver.implicitly_wait(5)
wait = WebDriverWait(driver, 10)

# Navigate to the upsrtc Bus Routes page
driver.get("https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile")
driver.maximize_window()
time.sleep(2)
route_data = []
# Function to scrape route data
def scrape_page():
    routescontainer = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[2]/div[1]/a')))
      
    for route in routescontainer:
        route_data.append({
            'Route Name': route.text,
            'Route Link': route.get_attribute('href')
        })
    return route_data  # Return the scraped data

# Pagination to collect routes from multiple pages
for page_number in range(1, 6):
    try:
        pagination_container = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))
        next_page_button = pagination_container.find_element(By.XPATH, f'.//div[contains(@class,"DC_117_pageTabs") and text()="{page_number + 1}"]')

        actions = ActionChains(driver)
        actions.move_to_element(next_page_button).perform()
        next_page_button.click()
        time.sleep(1)

        wait.until(EC.text_to_be_present_in_element((By.XPATH, f'.//div[contains(@class,"DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
        time.sleep(2)

        scrape_page()

    except Exception as e:
        print(e)
        break


# Function to scrape bus details
def collect_bus_details():
    alldetails = []
    for data in route_data:
        link = data['Route Link']
        route_name = data['Route Name']
        try:
            driver.get(link)
            time.sleep(2)
            try:
                view_buses_buttons = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[@class='button']")))
                for button in view_buses_buttons:
                    wait.until(EC.element_to_be_clickable(button)).click()
                    time.sleep(3)  # Wait for the buses to load
            except Exception as e:
                print(f"Error clicking 'View Buses' button for {route_name}: {e}")

            bus_containers = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, 'clearfix bus-item')]")))
            for bus_container in bus_containers:
                try:
                    busname = bus_container.find_element(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']").text
                    bustype = bus_container.find_element(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
                    departuretiming = bus_container.find_element(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']").text
                    duration = bus_container.find_element(By.XPATH, ".//div[@class='dur l-color lh-24']").text
                    reachtiming = bus_container.find_element(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']").text
                    rating = bus_container.find_element(By.XPATH, ".//div[@class='rating-sec lh-24']").text
                    price = bus_container.find_element(By.XPATH, ".//span[contains(@class,'f-19 f-bold') or contains(@class,'f-bold f-19')]").text
                    seat = bus_container.find_element(By.XPATH, ".//div[contains(@class, 'seat-left m-top-30') or contains(@class,'seat-left m-top-16')]").text

                    details = {
                        'State': "Kerala",
                        'Route Name': route_name,
                        'Route Link': link,
                        'Bus Name': busname,
                        'Bus Type': bustype,
                        'Departing Time': departuretiming,
                        'Duration': duration,
                        'Reaching Time': reachtiming,
                        'Star Rating': rating,
                        'Price': price,
                        'Seat Availability': seat,
                    }
                    # Check if the details already exist in the list
                    if details not in alldetails:
                        print(details)
                        alldetails.append(details)

                        # Insert data into the database
                        insert_query = """
                        INSERT IGNORE INTO BusDetails (state, route_name, route_link, bus_name, bus_type, departing_time, duration, reaching_time, star_rating, price, seat_availability)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                        """
                        cursor.execute(insert_query, (
                            details['State'],
                            details['Route Name'],
                            details['Route Link'],
                            details['Bus Name'],
                            details['Bus Type'],
                            details['Departing Time'],
                            details['Duration'],
                            details['Reaching Time'],
                            details['Star Rating'],
                            details['Price'],
                            details['Seat Availability']
                        ))
                        db_connection.commit()
                except Exception as e:
                    print(f"Error scraping bus details for {route_name}: {e}")
        except Exception as e:
            print(f"Error accessing route link {link}: {e}")
    return alldetails

# Main execution


bus_details = collect_bus_details()
print("Collected Bus Details:", bus_details)


df = pd.DataFrame(bus_details)
df.to_csv("Kerala.csv", index=False)

# Close the WebDriver
driver.quit()

# Close the Database Connection
cursor.close()
db_connection.close()

Database connected Successfully
Table created successfully
Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6CFDE02F5+28725]
	(No symbol) [0x00007FF6CFD42AE0]
	(No symbol) [0x00007FF6CFBD510A]
	(No symbol) [0x00007FF6CFC293D2]
	(No symbol) [0x00007FF6CFC295FC]
	(No symbol) [0x00007FF6CFC73407]
	(No symbol) [0x00007FF6CFC4FFEF]
	(No symbol) [0x00007FF6CFC70181]
	(No symbol) [0x00007FF6CFC4FD53]
	(No symbol) [0x00007FF6CFC1A0E3]
	(No symbol) [0x00007FF6CFC1B471]
	GetHandleVerifier [0x00007FF6D010F30D+3366989]
	GetHandleVerifier [0x00007FF6D01212F0+3440688]
	GetHandleVerifier [0x00007FF6D01178FD+3401277]
	GetHandleVerifier [0x00007FF6CFEAAAAB+858091]
	(No symbol) [0x00007FF6CFD4E74F]
	(No symbol) [0x00007FF6CFD4A304]
	(No symbol) [0x00007FF6CFD4A49D]
	(No symbol) [0x00007FF6CFD38B69]
	BaseThreadInitThunk [0x00007FFCCD9AE8D7+23]
	RtlUserThreadStart [0x00007FFCCE0FFBCC+44]

{'State': 'BIHAR', 'Route Name': 'Thrissur to Kozhikode', 'Route Link': 'https://www.redbus.in/bus-tickets/thrissur-